In [294]:
import pandas as pd
import seaborn as sns
import numpy as np
import pickle
sns.set()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [295]:
df = pd.read_pickle('/Users/andrewpeters/GitHub/fpl/data/interim/df_with_rolling_features.pkl')

In [296]:
df = df[df['total_points'] >= 0] #possible, but rare, to get negative points. Will remove negative points for ease of use
df['past_three_avg_mins'] = (df['minutes_prev'] + df['minutes_prev_2'] + df['minutes_prev_3'])/3
df = df[df['past_three_avg_mins'] > 60]
df = df[df['minutes'] > 40] #minutes played is very much after-the-fact knowledge, but usually we know who won't even be playing 40 minutes
df = df.drop(columns = ['past_three_avg_mins', 'minutes'])

In [297]:
def prep_final_features(df):
    df['was_home'] = df['was_home'].astype('int') #convert bool to numeric
    positions_df = pd.get_dummies(df['position'], prefix='position_')
    df = df.join(positions_df)
    return df

In [298]:
df = prep_final_features(df)

In [299]:
metadata_cols = ['player', 'team', 'position','opponent_team', 'gw', 'kickoff_time', 'season', 'team_opponent','total_points_opponent']

In [300]:
metadata = df[metadata_cols]

In [301]:
df = df.dropna()

In [302]:
X = df.drop(columns = metadata_cols)
X = X.drop(columns = 'total_points')

In [303]:
y = df['total_points']

In [304]:
#save the final features names -- I'll use this when predicting future data to make sure I have the correct features
with open('/Users/andrewpeters/GitHub/fpl/data/interim/x_features.pkl', 'wb') as f:
    pickle.dump(list(X), f)

In [305]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X) ##should move this to notebook 05, after making the train test split, to avoid data leakage

In [306]:
with open('/Users/andrewpeters/GitHub/fpl/data/processed/X.npy', 'wb') as file:
    np.save(file, X)

In [307]:
with open('/Users/andrewpeters/GitHub/fpl/data/processed/y.npy', 'wb') as file:
    np.save(file, y)

In [308]:
X.shape

(27669, 320)